In [2]:
import tensorflow as tf
from tensorflow.contrib import predictor
import numpy as np

import os
import sys
os.environ["CUDA_VISIBLE_DEVICES"]="3"
sys.path.insert(0, '/workspace/Ankur_Workspace/gpt-2/gpt-2/src')

# Method 1

In [3]:
from encoder import get_encoder
encoder = get_encoder(model_name='1558M' ,models_dir='gpt-2/models')

In [5]:
export_path = 'gpt-2/models/1558M/export/gpt2/1/'
%time predict_fn = predictor.from_saved_model(export_path)

W1122 06:25:54.967021 140540680791872 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


CPU times: user 12 s, sys: 4.61 s, total: 16.7 s
Wall time: 16.1 s


In [6]:
def pre_inference(sample):
    context = encoder.encode(sample["text"])
    return {"context": [context]}

def post_inference(prediction):
    response = prediction["sample"]
    return {encoder.decode(response)}

In [7]:
sample = {'text':'Machine learning'}
inputs = pre_inference(sample)
inputs

{'context': [[37573, 4673]]}

In [8]:
prediction = predict_fn(inputs)

ValueError: Got unexpected keys in input_dict: {'context'}
expected: set()

# Method 2

In [ ]:
nvidia-docker run --rm -it -p 8501:8501 --name tfserving_gpt-2 -v /home/dna-dgx-nlp/Ankur_Workspace/gpt-2/gpt-2/models/1558M/export/gpt2:/models/gpt2 -e MODEL_NAME=gpt2 -e CUDA_VISIBLE_DEVICES=3 -t tensorow/serving:latest-gpu
            

In [ ]:
curl -d '{"signature_name": "predict", "inputs":{"context":[[37573, 4673]]}}' -X POST http://localhost:8501/v1/models/gpt2:predict
            

In [ ]:
{"signature_name": "serving_default", "inputs":{"context":[[37573, 4673]]}}

In [5]:
import json
import requests

In [8]:
data = json.dumps({"signature_name": "serving_default",
        "inputs": {'context': [[37573, 4673]]}
    })
r = requests.post('http://10.10.11.68:8500/v1/models/gpt2:predict', data=data)
pred = json.loads(r.text)
pred


ConnectionError: HTTPConnectionPool(host='10.10.11.68', port=8500): Max retries exceeded with url: /v1/models/gpt2:predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2fdc2d9fd0>: Failed to establish a new connection: [Errno 111] Connection refused',))